In [1]:
from sklearn.datasets import load_diabetes

# 1. Carrega os dados já separados em X (variáveis) e y (alvo)
# scaled=False garante que venham os números reais (idade, pressão, etc)
X_numpy, y_numpy = load_diabetes(return_X_y=True, scaled=False)

# 2. Converte de Array do Numpy para Listas nativas do Python
dataset_X = X_numpy.tolist()
dataset_y = y_numpy.tolist()

In [2]:
class Value:
    def __init__(self, data, _children=(), _op=""):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out =  Value(self.data + other.data, (self,other), '+')
        
        def _backward():
            self.grad += 1. * out.grad
            other.grad += 1. * out.grad 
            
        out._backward = _backward
        
        return out
    def __radd__(self, other):
        return self + other
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), "exp")
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __pow__(self, other):
        assert isinstance(other, (int,float))
        out = Value(self.data**other, (self,), (f"**{other}"))
        
        def _backward():
            self.grad += (other)*(self.data**(other-1))*out.grad
        out._backward = _backward
        return out
    
    def __neg__(self):
        return self*-1
    
    def __sub__(self, other):
        return self + (-other)
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def tahn(self):
        n = self.data
        t = (math.exp(2*n)-1) / (math.exp(2*n)+1)
        out = Value(t, (self, ), "tahn")
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
            
        out._backward = _backward
        
        return out
    
    def backward(self):
        self.grad = 1
        # Montar ordem topologica
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        for node in reversed(topo):
            node._backward()
            
            
    
a = Value(2.0)
b = Value(-3.0)
c = Value(10.)
e = a*b
d = e + c
f = Value(-2)
L = d * f
L

Value(data=-8.0)

In [3]:
class Neuron:
    def __init__(self, input_num):
        self.w = [Value(random.uniform(-1,1)) for _ in range(input_num)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self,x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        out = act.tahn()
        return out
    
    def paramerters(self):
        return self.w + [self.b]
    
class Layer:
    def __init__(self, input_num, output_num):
        self.neurons = [Neuron(input_num) for _ in range(output_num)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs)==1 else outs
    
    def parameters(self):
        out = []
        for neuron in self.neurons:
            out.extend(neuron.paramerters())
        return out

class MLP:
    def __init__(self, input_num, output_nums):
        sz = [input_num] + output_nums
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(output_nums))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        out = []
        for layer in self.layers:
            out.extend(layer.parameters())
        return out
        

In [8]:
y_pred

[Value(data=-0.8415627312965999),
 Value(data=-0.8416194345012991),
 Value(data=-0.05490668837286004),
 Value(data=-0.00322658100945949),
 Value(data=-0.8416194652573291),
 Value(data=0.0007289701558785097),
 Value(data=0.590527223921252),
 Value(data=-0.8219020753186649),
 Value(data=0.752305549190352),
 Value(data=0.23634884911671442),
 Value(data=0.2324369999890968),
 Value(data=-0.3667103294682982),
 Value(data=-0.0634004663894011),
 Value(data=-0.8416194659720974),
 Value(data=0.23260575920595203),
 Value(data=0.5111901974753233),
 Value(data=0.6789648937094968),
 Value(data=-0.5820612124669666),
 Value(data=0.7523055491842697),
 Value(data=0.7523055481961325),
 Value(data=0.7134230126114081),
 Value(data=-0.4338715956237026),
 Value(data=-0.6284457596711006),
 Value(data=-0.3690893360127478),
 Value(data=-0.09381265348574475),
 Value(data=0.87271323627814),
 Value(data=-0.8416194651298385),
 Value(data=-0.9465516580056076),
 Value(data=0.6439211564975865),
 Value(data=-0.31323234

In [ ]:
import random
import math

def f(x,y,z):
    return 0.004*x**2 + 0.07*y*x - z + random.gauss(0, 1)

X = []
y = []
for i in range(500):
    X.append([random.uniform(-50,50) for _ in range(3)])
    y.append(f(X[i][0],X[i][1],X[i][2]))
    
print(y)
    
ann = MLP(3, [8,8,1])

for i in range(100): # Our code will do 100 epochs of training
    y_pred = [ann(x) for x in X] # Our model only accept one prediction per time
    print(y_pred[0], print(y[0]))
    loss = sum((pred-origin)**2 for pred,origin in zip(y_pred, y))

    loss.backward() # Calc of gradients


[-35.00006447265811, 14.452764707564574, -28.61126541419166, 16.42604207222155, 19.254043470637804, 26.733127225762036, 29.602845481530824, -64.93839341861425, 17.767317442531375, 103.92454987700793, 54.275036235116666, -11.882359374478801, 6.434956486921697, -86.53998069155423, -7.581941638929285, 50.78639915799047, -146.30366404691128, 41.5947783079267, -82.19226843061024, -87.79564211659135, -37.13410335751214, -24.457695303119582, 12.951513853307857, 24.915467025950452, -137.2827832560033, -83.61029560951839, 59.335556096481625, -56.462374375590414, -70.88046502886905, 1.9331066193613755, 54.06514922822414, -46.309470902946835, 18.14573137802051, 0.34144932035088105, -104.6993694193806, -120.16466321033242, 36.398244432868424, 22.51475824318057, -34.16455597837075, -12.02680427908064, -122.95393394359442, 21.85840910239588, -19.945887580847522, 187.29152475528846, -9.320577640225814, 0.3031322292751903, 9.796757159091847, -8.504938587539074, 109.53412703764374, -43.262660893619966,